Transport and Swimming Tracers
--

Define several tracers with swimming behavior:
 a. up 1 m/day
 b. up 5 m/day
 c. up 15 m/day
 
Initialize unit concentration everywhere, including all BCs.

Feed in Kd and insolation data.

After the run, compile maps of instantaneous growth potential, based on 
normalized vertical distribution integrated with light limitation function.

Complicating factors:
 - For buoyant tracers, depth-uniform initialization in the coastal ocean
   leads to high surface concentrations, which could then mix into the Bay.
   Practically, should be safe to assume that the time for that elevated
   concentration to be transported into the Bay is long relative to the time 
   for the vertical profile to come into balance.

A secondary run could/will create a series of releases at times of RS
scenes, and integrate until the next scene. Vertical distribution of release
is the tricky part. All at the surface? Following the distribution from the 
previous run, normalized to get match surface concentration to RS? Depth-uniform?


In [6]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")

In [7]:
from stompy import utils
import six
import shutil
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
import subprocess
import os

In [8]:
if 0:
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_with_temp"
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_with_temp")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_with_temp.hyd"))
if 0:
    # non-temperature run gets us 2 more days of output. actually no. valid data stops
    # 8/21 23:00,while the temperature runs goes to 8/23 00:00
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom"
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom.hyd"))

if 0: # longer, 16 layer run. So far delwaq1 crashes on this one. 
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_16layer"    
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_16layer")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_16layer.hyd"))
    
if 1: # longer? 10 layer? 
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_complete_take2"    
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_complete_take2")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_complete_take2.hyd"))

In [9]:
# wholesale copypasta https://github.com/rustychris/agg_wy2013/blob/master/tracer_tests/tracer_common.py
import logging
log = logging.getLogger()

import re
import matplotlib.pyplot as plt
from matplotlib import colors
from shapely import geometry, wkt
import stompy.plot.cmap as scmap
import xarray as xr
from stompy.grid import unstructured_grid
import numpy as np
import pandas as pd
import stompy.model.delft.waq_scenario as dwaq
from stompy.memoize import memoize
import logging as log
from stompy import utils
from stompy import filters
from stompy.plot import plot_wkb
import netCDF4
import warnings
from stompy.model.data_comparison import calc_metrics
import os
import six
from matplotlib import gridspec

cmap=scmap.load_gradient('turbo.cpt') # a less offensive 'jet'

In [11]:
def hydro_name(hydro): return "wy2022_take2"

def configure_dwaq(): # May be outdated, esp. for chicago.
    # configure DWAQ:
    DELFT_SRC="/opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64"
    #DELFT_SRC="/home/alliek/software/Delft3D-FM/64634"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE
    
    if 'LD_LIBRARY_PATH' in os.environ:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB+":"+os.environ['LD_LIBRARY_PATH']
    else:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB

configure_dwaq()
dfm_bin=os.path.join(os.environ['DELFT_SRC'],'bin')
waqpbexport=os.path.join(dfm_bin,'waqpbexport')
waqpbimport=os.path.join(dfm_bin,'waqpbimport')

class CommonSetup(object):
    """
    Common code for various tracers runs
    """
    name='common' # should overload
    hydro=None
    base_path=None # must be set!

    force=True # whether to allow re-using an existing run
    
    # start time offset from start of hydro by this delta
    # give it some decent spinup time
    start_time=np.datetime64("2022-08-10 00:00")
    
    # set length of the run. Appears to be the end of valid output, even though the
    # hydro reports going until the 25th.
    stop_time=np.datetime64("2022-08-22 23:00") # seems that even temperature run is dicey on last step
    
    integration_option="15.60" # if set, copied to WaqModel
    time_step=3000
    map_time_step=3000 # otherwise it will default to time_step, which could be really short.    

    waq_kws={}
    def __init__(self,**kw):
        utils.set_keywords(self,kw)   
        if self.base_path is None:
            self.base_path=self.calc_base_path()
                
            yyyymmdd=utils.to_datetime(self.start_time).strftime('%Y%m%d')
            self.base_path+="_%s"%(yyyymmdd)   
            
            # And make it unique on successive runs
            for seq in range(50):
                test_path=self.base_path
                if seq>0:
                    test_path+=f"-v{seq:03}"
                if not os.path.exists(test_path):
                    self.base_path=test_path
                    break
            else:
                raise Exception("Too many runs with same name")
            
            log.info("base_path defaults to %s"%self.base_path)
    def calc_base_path(self):
        p='run_%s_%s'%(hydro_name(self.hydro),self.name)
        return p
        
    def release_conc_3d(self,*a,**kw):
        C_2d=self.release_conc_2d(*a,**kw)
        C_3d=self.hydro.extrude_element_to_segment(C_2d)
        return C_3d

    def setup_model(self):
        # Create a WaqModel, add some tracers
        self.wm=wm=dwaq.WaqModel(hydro=self.hydro,
                                 overwrite=True,
                                 base_path=self.base_path,
                                 mon_time_step=1000000, # daily
                                 time_step=self.time_step,
                                 **self.waq_kws)
        # add some option for balances.
        wm.integration_option="%s BALANCES-OLD-STYLE BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT"%self.integration_option
        #wm.start_time+= self.start_offset
        wm.start_time = self.start_time # may have to be smarter about starting on an output time step.
        # hydro reports the wrong stop time. manually set.
        if self.stop_time is not None:
            wm.stop_time=self.stop_time
        
        self.setup_tracers()
        
        wm.parameters['ACTIVE_VertDisp']=1
        wm.parameters['ScaleVDisp']=1.0 
        
    def run_waq_model(self):
        assert self.base_path is not None,"Must specify base_path"
        
        if not self.force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                log.info("Run seems to exist -- will not run again")
                self.wm=dwaq.WaqModel.load(self.base_path,load_hydro=False)
                return

        self.setup_model()
        
        wm=self.wm
        wm.cmd_write_hydro()
        wm.cmd_write_inp()
        self.copy_notebook()        
        wm.cmd_delwaq1()
        wm.cmd_delwaq2()
        wm.cmd_write_nc()
    def copy_notebook(self):
        script_fn="run_transport_and_swimming_v00.ipynb"
        shutil.copyfile(script_fn,os.path.join(self.base_path,script_fn))
    def setup_tracer_continuity(self):
        # continuity tracer:
        self.wm.substances['continuity']=dwaq.Substance(initial=1.0)
        # This adds a concentration=1.0 boundary condition on all the boundaries.
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
        self.wm.add_bc(all_bcs,'continuity',1.0)
    


Custom processes
--

This is the low-level approach. There may be some Delft tools to automate this, but
my general experience is that the tools get out of date and become unsupported.

Each process needs two pieces:
 1. Fortran code that implements the operation
 2. Tables that associate a process name, inputs, and outputs with the fortran function.
 
The same fortran code is reused for similar processes. The tables can define multiple
entries that use the same fortran implementation.

This means there is the "simple" level of customizing processes (edit the tables), and
the "complete" level (write new fortran code, along with tables to use the new fortran
subroutine).

Compiling new fortran subroutines requires using a compiler similar to what was used to
compile the original delwaq library. I think Deltares tend to use intel compilers, and
intel compilers are generally compatible with gnu compilers, so this isn't necessarily
a major hurdle (and intel compilers are readily available for free now). Nonetheless,
getting everything to work with a bespoke fortran subroutine is more error-prone than
just editing the tables.

The tables are stored three different ways, which makes the editing process at first
confusing.
 - proc_def.def and proc_def.dat: binary (NEFIS), read by delwaq at runtime
 - proces.asc: the human (for some value of human) readable table
 - a bunch of CSV files
 
You'd want to just edit proces.asc, and generate proc_def.*, but no. You have to edit
proces.asc, convert that to CSV (`waqpbimport`), and then convert back (`waqpbexport`).
Also, even though `waqpbimport` updates the CSV files, it requires them to already
exist.

You'd expect that binary distribution of delwaq would come with the CSVs, but no.


So this process is tricky with a binary compile, even if you don't want to compile.


In [25]:
# Instantaneous release, just see how the blob moves.
# Rather than using "anonymous" tracers as in the age tracer code, easier 
# to use substances that already have settling defined, but with no
# other processes. Use AlgNN tracers since there are lots of them.

class SwimmingEverywhere(CommonSetup):
    swim_speeds=[0,-5.0,-15.0,-30,-50] # positive down.
    start_time=np.datetime64("2022-08-01 00:00")
    stop_time=np.datetime64("2022-08-31 00:00") 
    kd_path="../Kd_2022/Kd_sentinel3_1h/Kd_sent3_20220801_20220901.nc"
    
    def setup_tracers(self):
        self.add_light()
        
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]

        for swim_i,speed in enumerate(self.swim_speeds):
            #name=f'IM{swim_i+1}'
            name=f'Alg{swim_i+1:02d}'
            conc=f'BLOOM' + name
            # initial condition of 1.0
            unity=1.0
            self.wm.substances[conc]=dwaq.Substance(initial=unity)
            self.wm.parameters['VSed' + name]=  speed             
            self.wm.add_process('SED' + name)        
            
            # This adds a concentration=1.0 boundary condition on all the boundaries.
            self.wm.add_bc(all_bcs,conc,unity)

        self.wm.parameters['TaucS']=0.0 # no deposition - covers all algae.
        
    def add_light(self):
        if self.kd_path is None: return
        self.add_kd()
        self.wm.add_process(name='CalcRad')
        self.add_insolation()
        
        self.wm.map_output += ('RadSurf','Rad', 'RadBot','ExtVl')
        
    def add_insolation(self):
        cimis=xr.open_dataset('/richmondvol1/hpcshared/inputs/cimis/union_city-hourly-2022_bloom.nc')
        # Starts as PST, but the model is UTC.
        cimis=cimis.set_coords('time').swap_dims({'Date':'time'})
        cimis['time']=cimis['time']+np.timedelta64(8,'h')
        sol_rad=cimis['HlySolRad'].values
        sol_rad=utils.fill_invalid(sol_rad)

        t0=np.datetime64(self.hydro.time0)
        t_secs=((cimis.time.values-t0)/np.timedelta64(1,'s')).astype(np.int64)
        param=dwaq.ParameterTemporal(times=t_secs,values=sol_rad)
        self.wm.parameters['RadSurf'] = param
            
    def add_kd(self):
        # extrude to 3D, write seg function
        ds=xr.open_dataset(self.kd_path)
        g=unstructured_grid.UnstructuredGrid.read_ugrid(ds)
        assert np.allclose(g.cells_centroid(), self.hydro.grid().cells_centroid()),"Kd field grid does not match"

        t0=np.datetime64(self.hydro.time0)
        tsecs=(ds.time.values-t0)/np.timedelta64(1,'s')
        def seg_func(t):
            C_2d=ds['Kd'].sel(time=t0+t*np.timedelta64(1,'s'),method='nearest').values
            return self.hydro.extrude_element_to_segment(C_2d)
            
        # No self-shading, specify overall extinction directly
        self.wm.parameters['ExtVl'] = dwaq.ParameterSpatioTemporal(times=tsecs,func_t=seg_func)
        


In [26]:
if 1:
    pb=SwimmingEverywhere(hydro=hydro)

    pb.run_waq_model()

INFO:root:base_path defaults to run_wy2022_take2_common_20220801-v006
INFO:WaqModel: start time updated from hydro: 2022-05-01T00:00:00.000000
INFO:WaqModel: stop time update from hydro: 2022-10-01T00:00:00.000000
INFO:HydroFiles:Segment depth will be inferred
INFO:WaqModel:Parameters gleaned from hydro: NamedObjects([('surf', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f97fff29c40>), ('bottomdept', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f97fff7df70>), ('vertdisper', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2400>), ('tau', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2dc0>), ('temp', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2a60>), ('salinity', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2250>)])
INFO:WaqModel:Writing hydro data
INFO:HydroFiles:Using .bnd file, not writing out kludgey boundary-links.csv
INFO: